In [11]:
import numpy as np
import pandas as pd
from scipy import stats
from utils import get_data, save_data, get_table, get_predictions, add_intervals_to_test, all_tests
from plots import plot_error, plot_finish_groups, plot_interval_checks, plot_finish_age_gender
np.random.seed(2025)

race = "nyc"
size1, size2 = 2000, 20000
train_yr, test_yr = [2021, 2022, 2023], [2024]

# save_data(race_list=["bos", "nyc", "chi"], size_train=size1, size_test=size2, train_lis=train_yr, test_lis=test_yr)
train, test = get_data(racename=race, size_train=size1, size_test=size2, train_lis=train_yr, test_lis=test_yr, save=False)

In [12]:
# train_full, test_full = get_data(racename=race, size_train=size1, size_test=None, train_lis=train_yr, test_lis=test_yr, save=False)
# stats.ks_2samp(test["finish"], test_full["finish"]).pvalue

In [13]:
model_info = [
    ("M1", f"stan_results/model1/params_{race}.csv", ["alpha", "total_pace"]),
    ("M2", f"stan_results/model2/params_{race}.csv", ["alpha", "total_pace", "curr_pace"]),
    ("M3", f"stan_results/model3/params_{race}.csv", ["alpha", "total_pace", "curr_pace", "male", "age"]),
]
mpreds = {name: get_predictions(test, path, feats_lis=feats, full=False) for (name, path, feats) in model_info}
models, baseline = ["M1", "M2", "M3"], "BL"
test2 = get_table(test, mpreds, baseline_name=baseline)
test2

,id,dist,curr_pace,total_pace,finish,age,gender,year,male,malexage,alpha,lvl,BL,M1,M2,M3
20440,141403,5K,3.046923,3.046923,2.877847,34,M,2024,1,34,1,1,-13.560017,12.000674,12.009334,11.596794
53759,174722,5K,2.167317,2.167317,1.692200,32,W,2024,0,0,1,1,-91.103783,-54.241858,-54.023551,-60.736073
49096,170059,5K,2.211411,2.211411,2.019189,34,W,2024,0,0,1,1,-30.273683,5.790311,5.990237,0.334518
6227,127190,5K,3.385240,3.385240,3.432720,43,W,2024,0,0,1,1,2.873383,25.733819,25.709346,24.939130
43857,164820,5K,2.416626,2.416626,2.222193,33,W,2024,0,0,1,1,-25.461817,7.299552,7.429032,2.407826
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395505,132749,40K,2.799552,3.165058,3.146532,39,M,2024,1,39,1,8,-1.308163,-0.437521,-0.184188,-0.121120
404205,141449,40K,2.635741,2.877077,2.876474,27,W,2024,0,0,1,8,-0.051215,0.829033,0.670339,0.359697
434098,171342,40K,1.797268,1.955799,1.957732,26,M,2024,1,26,1,8,0.355058,1.060569,0.324834,0.099960
427045,164289,40K,2.068680,2.250605,2.241077,48,W,2024,0,0,1,8,-1.328444,-0.506035,-0.988264,-0.966818


In [14]:
tbl2 = all_tests(test2, [["BL", "M1"], ["M1", "M2"], ["M2", "M3"]], savename=f"analysis/tables/{race}_stattest.csv")
print(tbl2)
tbl = plot_error(test2, models, baseline, save_name=race, bar=True, other=True)

c_model = "M2"
a = plot_finish_groups(test2, model=c_model, baseline=baseline, save_name=race)
plot_finish_age_gender(test2, model=c_model, baseline=baseline, save_name=race, grouping="age")
tbl

/Users/brandononyejekwe/Documents/Personal_Projects/Marathon_Predictor/utils.py:227: UserWarning: p-value floored: true value smaller than 0.001. Consider specifying `method` (e.g. `method=stats.PermutationMethod()`.)
  value = stats.anderson_ksamp([arr1, arr2]).pvalue
/Users/brandononyejekwe/Documents/Personal_Projects/Marathon_Predictor/utils.py:227: UserWarning: p-value capped: true value larger than 0.25. Consider specifying `method` (e.g. `method=stats.PermutationMethod()`.)
  value = stats.anderson_ksamp([arr1, arr2]).pvalue


            KS Wilcoxon      CVM     AD
BL-M1  <0.0001  <0.0001  <0.0001  0.001
M1-M2  <0.0001  <0.0001  <0.0001  0.001
M2-M3   0.3635   0.0051   0.2349  0.250
File saved: analysis/plots/nyc_error_bar.png
File saved: analysis/tables/nyc_error.csv
File saved: analysis/plots/nyc_error_groups.png
File saved: analysis/plots/nyc_error_gender_age.png


,BL,M1,M2,M3,pcnt_M1,pcnt_M2,pcnt_M3
5K,21.426036,19.385018,19.396947,19.190398,0.095259,0.094702,0.104342
10K,20.436816,16.328429,15.666719,15.425312,0.201029,0.233407,0.245219
15K,19.057375,16.442928,13.248433,13.227870,0.137188,0.304813,0.305892
20K,16.879804,12.371108,9.745142,9.573729,0.267106,0.422674,0.432829
25K,11.999721,9.721874,7.117952,7.026119,0.189825,0.406824,0.414476
30K,9.178402,6.892563,4.726935,4.740253,0.249045,0.484994,0.483543
35K,4.945135,4.142006,2.577755,2.571042,0.162408,0.478729,0.480087
40K,1.155768,1.141675,0.752804,0.742932,0.012194,0.348655,0.357196
OVRL MAE,13.149000,10.819000,9.167000,9.076000,NaN,NaN,NaN
OVRL $R^2$,0.871000,0.927000,0.937000,0.937000,NaN,NaN,NaN


In [15]:
x

NameError: name 'x' is not defined

In [ ]:
mpreds2 = {name: (42195 / 60) / get_predictions(test, path, feats_lis=feats, full=True) for (name, path, feats) in model_info}
intervals_tbl = add_intervals_to_test(test2, mpreds2, models)
i_check, i_sizes = plot_interval_checks(intervals_tbl, models, save_name=race)

File saved: analysis/plots/chi_intervals
analysis/tables/chi_intsizes.csv
analysis/tables/chi_intcheck.csv


In [ ]:
param_lists = []
rnd = 4
d = pd.read_csv("stan_results/model2/params_bos.csv", index_col="Unnamed: 0")
params1 = d.describe().T["mean"]
params2 = d.describe().T["std"]
param_lists.append(params1[[f"beta.{i}.1" for i in range(1, 9)]].round(rnd).reset_index(drop=True).rename('alpha_mean'))
param_lists.append(params2[[f"beta.{i}.1" for i in range(1, 9)]].round(rnd).reset_index(drop=True).rename('alpha_std'))
param_lists.append(params1[[f"beta.{i}.2" for i in range(1, 9)]].round(rnd).reset_index(drop=True).rename('total_pace_mean'))
param_lists.append(params2[[f"beta.{i}.2" for i in range(1, 9)]].round(rnd).reset_index(drop=True).rename('total_pace_std'))
param_lists.append(params1[[f"beta.{i}.3" for i in range(1, 9)]].round(rnd).reset_index(drop=True).rename('curr_pace_mean'))
param_lists.append(params2[[f"beta.{i}.3" for i in range(1, 9)]].round(rnd).reset_index(drop=True).rename('curr_pace_std'))
param_lists.append(params1[[f"sigma.{i}" for i in range(1, 9)]].round(rnd).reset_index(drop=True).rename('sigma_mean'))
param_lists.append(params2[[f"sigma.{i}" for i in range(1, 9)]].round(rnd).reset_index(drop=True).rename('sigma_std'))
pd.concat(param_lists, axis=1).to_csv("analysis/tables/params_bos.csv")